# [자연어처리]
# 2주차(2-2). 카운트 기반의 문서 표현(1)



# 202002961 김현주
* 모든 셀 실행 후 제출하시기 바랍니다.
* **실습 (2-2-1)**이 있습니다.

## 1. BOW 기반의 카운트 벡터 생성

In [ ]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import movie_reviews
from nltk.tokenize import sent_tokenize, word_tokenize

print('#review count:', len(movie_reviews.fileids())) #영화 리뷰 문서의 id를 반환
print('#samples of file ids:', movie_reviews.fileids()[:10]) #id를 10개까지만 출력
print('#categories of reviews:', movie_reviews.categories()) # label, 즉 긍정인지 부정인지에 대한 분류
print('#num of "neg" reviews:', len(movie_reviews.fileids(categories='neg'))) #label이 부정인 문서들의 id를 반환
print('#num of "pos" reviews:', len(movie_reviews.fileids(categories='pos'))) #label이 긍정인 문서들의 id를 반환
fileid = movie_reviews.fileids()[0] #첫번째 문서의 id를 반환
print('#id of the first review:', fileid)
print('#first review content:\n', movie_reviews.raw(fileid)[:200]) #첫번째 문서의 내용을 200자까지만 출력
print()
print('#sentence tokenization result:', sent_tokenize(movie_reviews.raw(fileid))[:2]) #첫번째 문서를 sentence tokenize한 결과 중 앞 두 문장
print('#word tokenization result:', word_tokenize(movie_reviews.raw(fileid))[:20]) #첫번째 문서를 word tokenize한 결과 중 앞 스무 단어

#review count: 2000
#samples of file ids: ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']
#categories of reviews: ['neg', 'pos']
#num of "neg" reviews: 1000
#num of "pos" reviews: 1000
#id of the first review: neg/cv000_29416.txt
#first review content:
 plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
w

#sentence tokenization result: ['plot : two teen couples go to a church party , drink and then drive .', 'they get into an accident .']
#word tokenization result: ['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an']


**1단계**

각 문서에 대해 텍스트 전처리를 수행해 의미가 있는 최고 단위의 리스트로 변환 -> 토큰화, 정제

- 토큰화 :
여기서는 단순히 nltkd의 word_tokenize()를 사용하여 토큰화를 수행했다.

In [ ]:
documents = [word_tokenize(movie_reviews.raw(fileid)) for fileid in movie_reviews.fileids()]
print(documents[0][:50]) #첫째 문서의 앞 50개 단어를 출력

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'s", 'the', 'deal', '?', 'watch', 'the']


- 정제 : 정제를 적용하기 위해 word_count를 이용해 단어의 빈도수를 세고, 빈도수 상위 10개의 단어를 출력하도록 했다.

In [ ]:
word_count = {}
for text in documents:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1

sorted_features = sorted(word_count, key=word_count.get, reverse=True)
for word in sorted_features[:10]: # 빈도수 상위 10개의 단어 출력
    print(f"count of '{word}': {word_count[word]}", end=', ')

count of ',': 77717, count of 'the': 76276, count of '.': 65876, count of 'a': 37995, count of 'and': 35404, count of 'of': 33972, count of 'to': 31772, count of 'is': 26054, count of 'in': 21611, count of ''s': 18128, 

- 정제 : 불용어에 해당하지 않는 토큰과 길이가 2보다 큰 토큰만 추가하도록 하여 특성집합을 구성했다.

In [ ]:
from nltk.corpus import stopwords #일반적으로 분석대상이 아닌 단어들

english_stops = set(stopwords.words('english')) #영어 불용어를 가져옴

#words() 대신 raw()를 이용해 원문을 가져옴
documents = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]

# stopwords의 적용과 토큰화를 동시에 수행.
tokens = [[token for token in word_tokenize(doc) if token not in english_stops and len(token) > 2] for doc in documents]

word_count = {}
for text in tokens:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1

**2단계**

특성 추출 대상이 되는 단어 집합, 즉 특성 집합을 구성

In [ ]:
sorted_features = sorted(word_count, key=word_count.get, reverse=True)

print('num of features:', len(sorted_features))
for word in sorted_features[:10]:
    print(f"count of '{word}': {word_count[word]}", end=', ')

num of features: 45953
count of 'film': 9443, count of 'n't': 6217, count of 'movie': 5671, count of 'one': 5582, count of 'like': 3547, count of 'even': 2556, count of 'good': 2316, count of 'time': 2282, count of 'would': 2264, count of 'story': 2146, 

- 기준에 따라 단어들을 선별할 수 있는데, 여기에서는 빈도를 계산하고 빈도가 높은 단어 1000개만을 추출하여 문서를 표현할 특성으로 사용했다.

In [ ]:
word_features = sorted_features[:1000] #빈도가 높은 상위 1000개의 단어만 추출하여 features를 구성

**3단계**

각 문서별로 특성 추출 대상 단어들에 대해 단어의 빈도를 계산해 특성 벡터(카운터 벡터)를 추출

In [ ]:
def document_features(document, word_features):
    word_count = {}
    for word in document: #document에 있는 단어들에 대해 빈도수를 먼저 계산
        word_count[word] = word_count.get(word, 0) + 1

    features = []
    for word in word_features: #word_features의 단어에 대해 계산된 빈도수를 feature에 추가
        features.append(word_count.get(word, 0)) #빈도가 없는 단어는 0을 입력
    return features

word_features_ex = ['one', 'two', 'teen', 'couples', 'solo']
doc_ex = ['two', 'two', 'couples']
print(document_features(doc_ex, word_features_ex))

[0, 2, 0, 1, 0]


In [ ]:
feature_sets = [document_features(d, word_features) for d in tokens]

# 첫째 feature set의 내용을 앞 20개만 word_features의 단어와 함께 출력
for i in range(20):
    print(f'({word_features[i]}, {feature_sets[0][i]})', end=', ')

(film, 6), (n't, 6), (movie, 6), (one, 3), (like, 3), (even, 3), (good, 2), (time, 0), (would, 1), (story, 0), (much, 0), (character, 2), (also, 1), (get, 3), (characters, 1), (two, 2), (first, 0), (see, 2), (way, 2), (well, 1), 

- word_features의 단어는 빈도가 높은 순서대로 정렬되어 있기 때문에 위에서 20개 값을 출력해보면 모두 0으로 나오는 것을 확인할 수 있다.

In [ ]:
print(feature_sets[0][-20:])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 2. 사이킷런을 이용한 카운트 벡터 생성

### CountVectorizer

http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction  

In [ ]:
# data 준비, movie_reviews.raw()를 사용하여 raw text를 추출
reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#cv = CountVectorizer() #모든 매개변수에 디폴트 값을 사용하는 경우

#앞에서 생성한 word_features를 이용하여 특성 집합을 지정하는 경우
cv = CountVectorizer(vocabulary=word_features)

#cv = CountVectorizer(max_features=1000) #특성 집합을 지정하지 않고 최대 특성의 수를 지정하는 경우
print(cv) #객체에 사용된 인수들을 확인

CountVectorizer(vocabulary=['film', "n't", 'movie', 'one', 'like', 'even',
                            'good', 'time', 'would', 'story', 'much',
                            'character', 'also', 'get', 'characters', 'two',
                            'first', 'see', 'way', 'well', 'could', 'make',
                            'really', 'films', 'little', 'life', 'plot',
                            'people', 'scene', 'bad', ...])


In [ ]:
reviews_cv = cv.fit_transform(reviews) #reviews를 이용하여 count vector를 학습하고, 변환
print(cv.get_feature_names_out()[:20]) # count vector에 사용된 feature 이름을 반환
print(word_features[:20]) # 비교를 위해 출력

['film' "n't" 'movie' 'one' 'like' 'even' 'good' 'time' 'would' 'story'
 'much' 'character' 'also' 'get' 'characters' 'two' 'first' 'see' 'way'
 'well']
['film', "n't", 'movie', 'one', 'like', 'even', 'good', 'time', 'would', 'story', 'much', 'character', 'also', 'get', 'characters', 'two', 'first', 'see', 'way', 'well']


- 카운터 벡터의 형태가 2000x1000인 sparse matrix인 것을 알수 있다.
- 카운터 벡터의 대부분의 값이 0이므로 단어의 희소성으로 인해 (0, 0)과 같은 형태로 표현하면 공간을 효율적으로 사용할 수 있다.

In [ ]:
print('#type of count vectors:', type(reviews_cv))
print('#shape of count vectors:', reviews_cv.shape)
print('#sample of count vector:')
print(reviews_cv[0, :10])

#type of count vectors: <class 'scipy.sparse._csr.csr_matrix'>
#shape of count vectors: (2000, 1000)
#sample of count vector:
  (0, 0)	6
  (0, 2)	6
  (0, 3)	3
  (0, 4)	3
  (0, 5)	3
  (0, 6)	2
  (0, 8)	1


In [ ]:
reviews_cv

<2000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 253277 stored elements in Compressed Sparse Row format>

In [ ]:
for word, count in zip(cv.get_feature_names_out()[:20], reviews_cv[0].toarray()[0, :20]):
    print(f'{word}:{count}', end=', ')

film:6, n't:0, movie:6, one:3, like:3, even:3, good:2, time:0, would:1, story:0, much:0, character:2, also:1, get:3, characters:1, two:2, first:0, see:2, way:3, well:1, 

---

# **[[실습 2-2-1]]**
## 코드 셀 위에 이를 설명하는 **마크다운 텍스트를 3개 이상 추가**하시오. (포매팅을 하지는 않아도 됨)
* 코드 작동에 대한 설명 혹은 코드 결과에 대한 설명 등

예시)



### 특성의 값을 단어와 함께 보고 싶다면,

In [ ]:
for word, count in zip(cv.get_feature_names_out()[:20], reviews_cv[0].toarray()[0, :20]):
    print(f'{word}:{count}', end=', ')

film:6, n't:0, movie:6, one:3, like:3, even:3, good:2, time:0, would:1, story:0, much:0, character:2, also:1, get:3, characters:1, two:2, first:0, see:2, way:3, well:1, 

### 0번째 문서에 대해 20번째 feature 까지의 값을 살펴보았다.
* film은 6번 등장하였고, n't는 0번 등장하였다. 이처럼 feature로 사용된 단어가 몇 번 등장했는지 횟수가 담긴 matrix가 만들어졌음을 알 수 있다.

---

# [파일] -> [다운로드] -> [.ipynb 다운로드]